In [1]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument("headless")

In [2]:
driver_path = 'C:/Users/MY/Downloads/chromedriver.exe'

In [3]:
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', cleaned_text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [13]:
def biz_khan(query,driver_path,page):
    domain = '한국일보'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
    
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    head_list = []
    data_list = []
    
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path = driver_path)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain = '%EB%89%B4%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1469&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        
        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
                
        except:
            print('기사가 없')
            driver.close()
            return 0
        driver.close()
        
        
        for url in url_list:
            driver = webdriver.Chrome(executable_path=driver_path)  # for Windows
            try:
                driver.get(url)
                time.sleep(1)
            
            except:
                print('link error')
                continue
                # try문 안에서 각자 언론사에 맞게 수정
                
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(clean_text(soup.find('h2').text))
                tags = soup.select_one('body > div.wrap > div.container.end.end-uni > div.end-body > div > div.col-main.read').find_all('p')
                data = [x.text for x in tags]
                data_list.append(data)
            
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page+=10
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

biz = biz_khan('풀무원 바른먹거리',driver_path,5)
biz

,언론사,제목,내용
0,한국일보,냉면·짜장·떡볶이 제품에 트레이가 필요한가요,"[[쓰레기를 사지 않을 권리]<15>즉석조리식품 트레이, 기후위기와 쓰레기산에 신음..."
1,한국일보,풀무원 입사 1호 사원이 … 성장세 발목 잡던 해외 실적도 개선,"[\n원혜영 의원 부친 농장, 남승우 前대표가 기업화\n, \n‘바른 먹거리’로 年..."
2,한국일보,풀무원 13년 만에 기업이미지 바꿔… 계열사 브랜드도 통일,[풀무원이 2005년 이후 13년 만에 기업이미지(CI)를 교체하고 계열사 브랜드 ...
3,한국일보,위원회 출범하고 도…식품업계 경영 속도,[기업의 비(非)재무적 성과를 뜻하는 ESG(환경·사회·지배구조) 경영이 유통업계의...
4,한국일보,위원회 출범하고 도…식품업계 경영 속도,[기업의 비(非)재무적 성과를 뜻하는 ESG(환경·사회·지배구조) 경영이 유통업계의...
5,한국일보,위원회 출범하고 도…식품업계 경영 속도,[기업의 비(非)재무적 성과를 뜻하는 ESG(환경·사회·지배구조) 경영이 유통업계의...
6,한국일보,위원회 출범하고 도…식품업계 경영 속도,[기업의 비(非)재무적 성과를 뜻하는 ESG(환경·사회·지배구조) 경영이 유통업계의...


In [10]:
biz.to_csv('한국일보.csv',encoding='utf-8-sig', index = None)

In [11]:
data1 = pd.read_csv('한국일보.csv',encoding='utf-8-sig')

In [12]:
data1

,언론사,제목,내용
0,한국일보,냉면·짜장·떡볶이 제품에 트레이가 필요한가요,"['[쓰레기를 사지 않을 권리]<15>즉석조리식품 트레이', ""기후위기와 쓰레기산에..."
1,한국일보,풀무원 입사 1호 사원이 … 성장세 발목 잡던 해외 실적도 개선,"['\n원혜영 의원 부친 농장, 남승우 前대표가 기업화\n', '\n‘바른 먹거리’..."
2,한국일보,풀무원 13년 만에 기업이미지 바꿔… 계열사 브랜드도 통일,['풀무원이 2005년 이후 13년 만에 기업이미지(CI)를 교체하고 계열사 브랜드...
3,한국일보,위원회 출범하고 도…식품업계 경영 속도,['기업의 비(非)재무적 성과를 뜻하는 ESG(환경·사회·지배구조) 경영이 유통업계...
4,한국일보,위원회 출범하고 도…식품업계 경영 속도,['기업의 비(非)재무적 성과를 뜻하는 ESG(환경·사회·지배구조) 경영이 유통업계...
5,한국일보,위원회 출범하고 도…식품업계 경영 속도,['기업의 비(非)재무적 성과를 뜻하는 ESG(환경·사회·지배구조) 경영이 유통업계...
6,한국일보,위원회 출범하고 도…식품업계 경영 속도,['기업의 비(非)재무적 성과를 뜻하는 ESG(환경·사회·지배구조) 경영이 유통업계...
